In [1]:
import pandas as pd
import numpy as np
import itertools
import collections

In [2]:
class LinkedList:
    def __init__(self):
        self.head = None
        self.work = 0
        
    def push(self,item):
        self.head = [item,self.head]
        self.work += 1
        
    def pop(self):
        popped,self.head = self.head
        self.work += 1
        return popped
    
    def insert(self,item,index):
        self.work += 1
        if index == 0:
            self.head = (item,self.head)
        else:
            parent_node = self.get(index-1)
            parent_node[1] = [item,parent_node[1]]
            
                
    def get(self,index):
        node = self.head
        self.work += 1
        for i in xrange(index):
            self.work += 1
            _,node = node
        return node
    
    def _search(self, item):
        node= self.head
        self.work += 1
        index = 0
        while node is not None:
            if node[0] == item:
                return node,index
            _,node = node
            index += 1
        return None,None
    
    def search_index(self, item):
        node,index = self._search(item)
        
    def search_node(self, item):
        node,index = self._search(item)
    
    def remove(self, index):
        if index == 0:
            self.work += 1
            _, self.head = self.head
        else:
            parent_node = self.get(index-1)
            parent_node[1] = parent_node[1][1]
            
    def append(self,item):
        if self.head == None:
            self.push(item)
        else:
            node = self.head
            while node[1] is not None:
                _,node = node
                self.work += 1
            node[1] = [item,None]
        self.work += 1
    
    def __repr__(self):
        node = self.head
        items = []
        while node is not None:
            item,node = node
            items.append(str(item))
        return ' -> '.join(items)

In [3]:
a = LinkedList()
a.append(1)
a.append(2)
a.append(3)
a.get(0)
a.insert(5,2)
a.remove(2)
a

1 -> 2 -> 3

# Part 2

## Move to Front
This technique moves the element which is accessed to the head of the list.

In [10]:
class MoveToFrontList(LinkedList):
    def move_to_front(self,item):
        node = self.head
        if node[0] == item:
            return node[0]
        while node is not None:
            if node[0] == item:
                parent[1] = node[1]
                self.push(node[0])
                return node[0]
            parent = node
            node = node[1]
            

In [21]:
a = MoveToFrontList()
a.append(1)
a.append(2)
a.append(3)
a

1 -> 2 -> 3

In [22]:
a.move_to_front(3)
a

3 -> 1 -> 2

In [23]:
a.move_to_front(2)
a

2 -> 3 -> 1

In [24]:
a.move_to_front(3)
a

3 -> 2 -> 1

## Count Method
In this technique, the number of times each node was searched for is counted i.e. every node keeps a separate counter variable which is incremented every time it is called. The nodes are then rearranged according to decreasing count.

In [68]:
class CountList(LinkedList):
        
    def search(self, item):
        node= self.head
        self.work += 1
        index = 0
        while node is not None:
            if node[0][0] == item:
                node[0][1] += 1
                self.remove(index)
                self.add(node[0])
                return 
            _,node = node
            index += 1
        
    def add(self, new_node):
        node = self.head
        if node is None or new_node[1] > node[0][1]:
            self.head = [new_node,node]
            return
        while node[1] is not None:
            parent = node
            _,node = node   
            if new_node[1] > node[0][1]:
                parent[1] = [new_node,parent[1]]
                #print 'Adding {} after {}'.format(str(new_node),str(parent[0]))
                return
        #print 'Adding {} at end'.format(str(new_node))
        node[1] = [new_node,None]

In [95]:
a = CountList()
a.add([1,0])
a.add([2,0])
a.add([3,0])
print 'Start:         ',a
a.search(3)
print 'Search 3:      ',a
a.search(3)
print 'Search 3 again:',a
a.search(2)
print 'Search 2:      ',a
a.search(2)
print 'Search 2 again:',a
a.search(2)
print 'Search 2 again:',a

Start:          [1, 0] -> [2, 0] -> [3, 0]
Search 3:       [3, 1] -> [1, 0] -> [2, 0]
Search 3 again: [3, 2] -> [1, 0] -> [2, 0]
Search 2:       [3, 2] -> [2, 1] -> [1, 0]
Search 2 again: [3, 2] -> [2, 2] -> [1, 0]
Search 2 again: [2, 3] -> [3, 2] -> [1, 0]


## Transpose Method
This technique involves swapping an accessed node with its predecessor. Therefore, if any node is accessed, it is swapped with the node in front unless it is the head node, thereby increasing its priority. 

In [96]:
class TransposeList(LinkedList):
    def search(self,item):
        node = self.head
        if node[0] == item:
            return node
        while node is not None:
            if node[0] == item:
                parent[0],node[0] = node[0],parent[0]
                return parent
            parent = node
            _,node = node             

In [102]:
a = TransposeList()
a.append(1)
a.append(2)
a.append(3)
print 'Start:   ',a
a.search(3)
print 'Search 3:',a
a.search(3)
print 'Search 3:',a
a.search(1)
print 'Search 1:',a

Start:    1 -> 2 -> 3
Search 3: 1 -> 3 -> 2
Search 3: 3 -> 1 -> 2
Search 1: 1 -> 3 -> 2
